# Arithmetik

Eine wichtige Funktion von pandas ist das arithmetische Verhalten bei Objekten mit unterschiedlichen Indizes. Wenn beim Addieren von Objekten die Indexpaare nicht gleich sind, wird der entsprechende Index im Ergebnis die Vereinigung der Indexpaare sein. Für Benutzer mit Datenbankerfahrung ist dies vergleichbar mit einem automatischen [OUTER JOIN](https://de.wikipedia.org/wiki/Join_(SQL)#%C3%84u%C3%9Ferer_Verbund_(OUTER_JOIN)) auf den Indexbezeichnungen. Schauen wir uns ein Beispiel an:

In [1]:
import numpy as np
import pandas as pd


s1 = pd.Series(np.random.randn(5))
s2 = pd.Series(np.random.randn(7))

Addiert man diese Werte, erhält man:

In [2]:
s1 + s2

0   -3.583041
1    1.307006
2    1.075852
3    1.041123
4    2.397227
5         NaN
6         NaN
dtype: float64

Der interne Datenabgleich führt zu fehlenden Werten an den Stellen der Labels, die sich nicht überschneiden. Fehlende Werte werden dann bei weiteren arithmetischen Berechnungen weitergegeben.

Bei DataFrames wird die Ausrichtung sowohl für die Zeilen als auch für die Spalten durchgeführt:

In [3]:
df1 = pd.DataFrame(np.random.randn(5,3))
df2 = pd.DataFrame(np.random.randn(7,2))

Wenn die beiden DataFrames addiert werden, ergibt sich ein DataFrame, dessen Index und Spalten die Vereinigungen derjenigen in jedem der obigen DataFrames sind:

In [4]:
df1 + df2

,0,1,2
0,-1.249554,1.414471,NaN
1,-1.152496,-0.526046,NaN
2,-0.563285,-1.919667,NaN
3,-0.818103,-1.213753,NaN
4,-1.267942,0.887387,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN


Da die Spalte 2 nicht in beiden DataFrame-Objekten vorkommen, erscheinen sie im Ergebnis als fehlend. Das Gleiche gilt für die Zeilen, deren Bezeichnungen nicht in beiden Objekten vorkommen.

## Arithmetische Methoden mit Füllwerten

Bei arithmetischen Operationen zwischen unterschiedlich indizierten Objekten kann es sinnvoll sein, einen speziellen Wert (z. B. `0`) zu verwenden, wenn eine Achsenbeschriftung in einem Objekt gefunden wird, im anderen aber nicht.  Mit der `add`-Methode kann das Argument `fill_value` übergeben werden:

In [5]:
df12 = df1.add(df2, fill_value=0)

df12

,0,1,2
0,-1.249554,1.414471,-0.926853
1,-1.152496,-0.526046,-1.331257
2,-0.563285,-1.919667,-2.501289
3,-0.818103,-1.213753,0.577517
4,-1.267942,0.887387,0.109231
5,0.233121,0.397626,NaN
6,-0.853883,-1.058706,NaN


Im folgenden Beispiel setzen wir die beiden verbleibenden NaN-Werte auf `0`:

In [6]:
df12.iloc[[5,6], [2]] = 0

In [7]:
df12

,0,1,2
0,-1.249554,1.414471,-0.926853
1,-1.152496,-0.526046,-1.331257
2,-0.563285,-1.919667,-2.501289
3,-0.818103,-1.213753,0.577517
4,-1.267942,0.887387,0.109231
5,0.233121,0.397626,0.000000
6,-0.853883,-1.058706,0.000000


## Arithmetische Methoden

Methode | Beschreibung
:------ | :-----------
`add`, `radd` | Methoden für Addition (`+`)
`sub`, `rsub` | Methoden für die Subtraktion (`-`)
`div`, `rdiv` | Methoden für die Division (`/`)
`floordiv`, `rfloordiv` | Methoden für die Abrundungsfunktion (engl.: floor divison) (`//`)
`mul`, `rmul` | Methoden für die Multiplikation (`*`)
`pow`, `rpow` | Methoden zur Potenzierung (`**`)

`r` (engl.: _reverse_) kehrt die Methode um.

## Operationen zwischen DataFrame und Series

Wie bei NumPy-Arrays verschiedener Dimensionen ist auch die Arithmetik zwischen DataFrame und Series definiert.

In [8]:
s1 + df12

,0,1,2,3,4
0,-1.883961,3.221520,-1.603931,NaN,NaN
1,-1.786903,1.281004,-2.008336,NaN,NaN
2,-1.197692,-0.112617,-3.178368,NaN,NaN
3,-1.452509,0.593297,-0.099561,NaN,NaN
4,-1.902348,2.694437,-0.567848,NaN,NaN
5,-0.401286,2.204675,-0.677079,NaN,NaN
6,-1.488290,0.748343,-0.677079,NaN,NaN


Wenn wir `s1` mit `df12` addieren, wird die Additon für jede Zeile einmal durchgeführt. Dies wird als _Broadcasting_ bezeichnet. Standardmäßig entspricht die Arithmetik zwischen DataFrame und Serie dem Index der Serie in den Spalten des DataFrame, wobei die Zeilen nach unten übertragen werden.

Wenn ein Indexwert weder in den Spalten des DataFrame noch im Index der Serie gefunden wird, werden die Objekte neu indiziert, um die Vereinigung zu bilden:

Wenn ihr stattdessen die Spalten übertragen und die Zeilen abgleichen wollt, müsst ihr eine der arithmetischen Methoden verwenden, z.B.:

In [9]:
df12.add(s2, axis="index")

,0,1,2
0,-4.198189,-1.534164,-3.875487
1,-1.652540,-1.026089,-1.831301
2,1.189646,-0.166736,-0.748358
3,0.104381,-0.291268,1.500002
4,-0.048569,2.106760,1.328604
5,-1.142244,-0.977739,-1.375365
6,-0.464831,-0.669654,0.389053


Die Achsennummer, die ihr übergebt, ist die Achse, auf die abgeglichen werden soll. In diesem Fall soll der Zeilenindex des DataFrame (`axis='index'` oder `axis=0`) abgeglichen und übertragen werden.

## Funktionsanwendung und Mapping

`numpy.ufunc` (elementweise Array-Methoden) funktionieren auch mit Pandas-Objekten:

In [10]:
np.abs(df12)

,0,1,2
0,1.249554,1.414471,0.926853
1,1.152496,0.526046,1.331257
2,0.563285,1.919667,2.501289
3,0.818103,1.213753,0.577517
4,1.267942,0.887387,0.109231
5,0.233121,0.397626,0.000000
6,0.853883,1.058706,0.000000


Eine weitere häufige Operation ist die Anwendung einer Funktion auf eindimensionale Arrays auf jede Spalte oder Zeile. Die [pandas.DataFrame.apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html)-Methode tut genau dies:

In [11]:
df12

,0,1,2
0,-1.249554,1.414471,-0.926853
1,-1.152496,-0.526046,-1.331257
2,-0.563285,-1.919667,-2.501289
3,-0.818103,-1.213753,0.577517
4,-1.267942,0.887387,0.109231
5,0.233121,0.397626,0.000000
6,-0.853883,-1.058706,0.000000


In [12]:
f = lambda x: x.max() - x.min()

df12.apply(f)

0    1.501062
1    3.334137
2    3.078807
dtype: float64

Hier wird die Funktion `f`, die die Differenz zwischen dem Maximum und dem Minimum einer Reihe berechnet, einmal für jede Spalte des Rahmens aufgerufen. Das Ergebnis ist eine Reihe mit den Spalten des Rahmens als Index.

Wenn ihr `axis='columns'` an `apply` übergebt, wird die Funktion stattdessen einmal pro Zeile aufgerufen:

In [13]:
df12.apply(f, axis="columns")

0    2.664025
1    0.805211
2    1.938004
3    1.791270
4    2.155329
5    0.397626
6    1.058706
dtype: float64

Viele der gebräuchlichsten Array-Statistiken (wie `sum` und `mean`) sind DataFrame-Methoden, so dass die Verwendung von `apply` nicht notwendig ist.

Die an apply übergebene Funktion muss keinen Einzelwert zurückgeben; sie kann auch eine Reihe mit mehreren Werten zurückgeben:

In [14]:
def f(x):
    return pd.Series([x.min(), x.max()], index=["min", "max"])


df12.apply(f)

,0,1,2
min,-1.267942,-1.919667,-2.501289
max,0.233121,1.414471,0.577517


Es können auch elementweise Python-Funktionen verwendet werden. Angenommen, ihr möchtet aus jedem Fließkommawert in `df12` eine formatierte Zeichenkette berechnen. Dies könnt ihr mit [pandas.DataFrame.applymap](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.applymap.html) erreichen:

In [15]:
f = lambda x: round(x, 2)

df12.map(f)

,0,1,2
0,-1.25,1.41,-0.93
1,-1.15,-0.53,-1.33
2,-0.56,-1.92,-2.50
3,-0.82,-1.21,0.58
4,-1.27,0.89,0.11
5,0.23,0.40,0.00
6,-0.85,-1.06,0.00


Der Grund für den Namen `applymap` ist, dass Series über eine `map`-Methode zur Anwendung einer elementweisen Funktion verfügt:

In [16]:
df12[2].map(f)

0   -0.93
1   -1.33
2   -2.50
3    0.58
4    0.11
5    0.00
6    0.00
Name: 2, dtype: float64